# AWS Glue Job - Load Commodities data to Silver Stage from API source


## Set the Glue session parameters


In [8]:
%iam_role arn:aws:iam::212430227630:role/LabRole
%region us-east-1
%number_of_workers 2

%idle_timeout 30
%glue_version 4.0
%worker_type G.1X

%%configure
{
  "--enable-metrics": "true",
  "--enable-observability-metrics": "true"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current iam_role is arn:aws:iam::212430227630:role/LabRole
iam_role has been set to arn:aws:iam::212430227630:role/LabRole.
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Previous number of workers: None
Setting new number of workers to: 2
Current idle_timeout is None minutes.
idle_timeout has been set to 30 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
The following configurations have been updated: {'--enable-metrics': 'true', '--enable-observability-metrics': 'true'}


##  Set up and start your interactive session.


In [11]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import boto3

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 30
Session ID: e0913d7a-e6b6-4b92-8e72-aa8a5c4748e6
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
--enable-metrics true
--enable-observability-metrics true
Waiting for session e0913d7a-e6b6-4b92-8e72-aa8a5c4748e6 to get into ready status...
Session e0913d7a-e6b6-4b92-8e72-aa8a5c4748e6 has been created.



## Transform Bronze data to Silver - COMMODITIES


In [2]:
from datetime import datetime, timedelta, timezone
from functools import reduce

import pyspark.sql.functions as F

### Set AWS Storage parameters


In [3]:
BUCKET_NAME = "cryptoengineer"
BRONZE_PREFIX = "datalake/bronze/commodities"

### Load job parameters

In [4]:
glue_client = boto3.client("glue")
# Check if params come from a GLUE workflow
if '--WORKFLOW_NAME' in sys.argv and '--WORKFLOW_RUN_ID' in sys.argv:
    print("Running in Glue Workflow")
    
    glue_args = getResolvedOptions(
        sys.argv, ['WORKFLOW_NAME', 'WORKFLOW_RUN_ID']
    )
    
    print("Reading the workflow parameters")
    workflow_args = glue_client.get_workflow_run_properties(
        Name=glue_args['WORKFLOW_NAME'], RunId=glue_args['WORKFLOW_RUN_ID']
    )["RunProperties"]

    
    time_frame = int(workflow_args['time_frame'])
    symbols = workflow_args['symbols']

else:
    # Check if params come from a Glue Job
    try:
        args = getResolvedOptions(sys.argv,
                                  ['JOB_NAME',
                                   'time_frame',
                                   'symbols'
                                   ])

        time_frame = int(args['time_frame'])
        symbols = args['symbols']
        print("Running as Job")
    except:
        print("Running as an interactive session")
        time_frame = 24
        symbols = "CLUSD,GCUSD,NGUSD"


Running as an interactive session


In [5]:
print("Time Frame: ", time_frame)
print("Symbols: ", symbols)

Time Frame:  408
Symbols:  CLUSD,GCUSD,NGUSD


#### Set the start and end dates for the data you want to load

In [6]:
# Start date
start_date = (datetime.utcnow() - timedelta(hours=time_frame)).strftime("%Y-%m-%d")
end_date = datetime.utcnow().strftime("%Y-%m-%d")

print("Start date; ",start_date," End date: ",end_date)

Start date;  2024-09-07  End date:  2024-09-24


## Read max date loaded to Silver from the INFO table

In [7]:
PREFIX_INFO_TABLE='datalake/gold/commodities'
path=f"s3://{BUCKET_NAME}/{PREFIX_INFO_TABLE}"
print("Path:", path)

Path: s3://cryptoengineer/datalake/gold/commodities


In [8]:
df_max_dates_silver = (
    spark
    .read
    .parquet(path)
    .filter(F.col("stage")=='silver')
    .select('symbol','base_currency','frequencies','end_datetime')
    .toPandas()
)

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [9]:
df_max_dates_silver.head(10)

  symbol base_currency frequencies        end_datetime
0  BZUSD           USD       15min 2024-09-24 11:00:00
1  BZUSD           USD        1day 2024-09-24 00:00:00
2  CLUSD           USD       15min 2024-09-06 16:45:00
3  CLUSD           USD        1day 2024-09-06 00:00:00
4  GCUSD           USD       15min 2024-09-06 16:45:00
5  GCUSD           USD        1day 2024-09-06 00:00:00
6  NGUSD           USD       15min 2024-09-06 16:45:00
7  NGUSD           USD        1day 2024-09-06 00:00:00


## Load the Bronze/Raw data for the time frame and symbol

In [10]:
path=f"s3://{BUCKET_NAME}/{BRONZE_PREFIX}"
print("Path:",path)

Path: s3://cryptoengineer/datalake/bronze/commodities


In [11]:
df= (
    spark
    .read
    .parquet(path)
    .filter(F.col("load_date").between(start_date, end_date))
)

In [12]:
print("Records: ", df.count())

Records:  4603


In [14]:
"""
(
    df
    .groupBy('source','type','symbol', 'year','base_currency', 'frequency')
    .count().alias('count')
    #.filter(F.col('frequency') == "15min")
    #.filter(F.col('type') == "FOREX")
    .orderBy('symbol','year')
    .show(50)
)
"""

+------+-----------+------+----+-------------+---------+-----+
|source|       type|symbol|year|base_currency|frequency|count|
+------+-----------+------+----+-------------+---------+-----+
|   FMP|COMMODITIES| BZUSD|2024|          USD|    15min|  156|
|   FMP|COMMODITIES| BZUSD|2024|          USD|     1day|    2|
|   FMP|COMMODITIES| CLUSD|2024|          USD|    15min|  160|
|   FMP|COMMODITIES| CLUSD|2024|          USD|     1day|    2|
|   FMP|COMMODITIES| GCUSD|2024|          USD|    15min|  160|
|   FMP|COMMODITIES| GCUSD|2024|          USD|     1day|    2|
|   FMP|COMMODITIES| NGUSD|2024|          USD|    15min|  160|
|   FMP|COMMODITIES| NGUSD|2024|          USD|     1day|    2|
+------+-----------+------+----+-------------+---------+-----+


In [13]:
## Get symbol, base and freq to read
df_symbols =(
    df
    .groupBy('symbol','base_currency','frequency')
    .count()
    .toPandas()
)

In [14]:
df_symbols.head(10)

  symbol base_currency frequency  count
0  CLUSD           USD     15min   1160
1  GCUSD           USD     15min   1160
2  NGUSD           USD     15min   1160
3  BZUSD           USD     15min   1059
4  GCUSD           USD      1day     16
5  CLUSD           USD      1day     16
6  NGUSD           USD      1day     16
7  BZUSD           USD      1day     16


## Extract only data not loaded into Silver

In [15]:
df_list=[]
for ind in df_symbols.index:
    print("Symbol: ", df_symbols['symbol'][ind], " Base: ", df_symbols['base_currency'][ind], " Frequency: ", df_symbols['frequency'][ind])
    # Get max date loaded to silver
    max_datetime=df_max_dates_silver[(df_max_dates_silver['symbol'] ==df_symbols['symbol'][ind])
                              & (df_max_dates_silver['base_currency'] ==df_symbols['base_currency'][ind])
                              & (df_max_dates_silver['frequencies'] ==df_symbols['frequency'][ind])]['end_datetime'].max()
    # Select the bronze data loaded to bronze where datetime > max datetime
    df_new_data= (
        df
        .filter(F.col('symbol')==df_symbols['symbol'][ind])
        .filter(F.col('base_currency')==df_symbols['base_currency'][ind])
        .filter(F.col('frequency')==df_symbols['frequency'][ind])
        .filter(F.col('datetime')>max_datetime)
    )
    print("Records: ", df_new_data.count())
    df_list.append(df_new_data)

Symbol:  CLUSD  Base:  USD  Frequency:  15min
Records:  1092
Symbol:  GCUSD  Base:  USD  Frequency:  15min
Records:  1092
Symbol:  NGUSD  Base:  USD  Frequency:  15min
Records:  1092
Symbol:  BZUSD  Base:  USD  Frequency:  15min
Records:  0
Symbol:  GCUSD  Base:  USD  Frequency:  1day
Records:  15
Symbol:  CLUSD  Base:  USD  Frequency:  1day
Records:  15
Symbol:  NGUSD  Base:  USD  Frequency:  1day
Records:  15
Symbol:  BZUSD  Base:  USD  Frequency:  1day
Records:  0


In [16]:
# Reduce and union al dataframes
df = reduce(lambda x, y: x.union(y), df_list)
print("Rows: ", df.count())

Rows:  3321


In [17]:
df.printSchema()

root
 |-- datetime: string (nullable = true)
 |-- open: double (nullable = true)
 |-- low: double (nullable = true)
 |-- high: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- date: string (nullable = true)
 |-- base_currency: string (nullable = true)
 |-- source: string (nullable = true)
 |-- frequency: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- audit_time: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- load_date: date (nullable = true)


## Remove and filter values

In [20]:
"""
(
    df
    .groupBy('symbol','year','base_currency', 'datetime','source','frequency','type')
    .count().alias('count')
    .filter(F.col('count') > 1)
    .orderBy('count', ascending=False)
    .show(100)
)
"""

+------+----+-------------+--------+------+---------+----+-----+
|symbol|year|base_currency|datetime|source|frequency|type|count|
+------+----+-------------+--------+------+---------+----+-----+
+------+----+-------------+--------+------+---------+----+-----+


In [18]:
df = (
    df
    .dropDuplicates(['symbol','year','datetime','source','frequency','type'])
)

In [19]:
print("After dropDuplicates: ", df.count())

After dropDuplicates:  3321


## Change Schema and uppercase column names

In [20]:
df = (
    df
    .withColumn("datetime", F.when(F.col("datetime").rlike("^\d{4}-\d{2}-\d{2}$"), F.to_timestamp(F.concat(F.col("datetime"), F.lit(" 00:00:00"))))
                            .otherwise(F.to_timestamp(F.col("datetime"))))
    .withColumn('volume', F.col('volume').cast('double'))
    .withColumn('month', F.col('month').cast('int'))
    .withColumn('day', F.col('day').cast('int'))
)

In [21]:
df = df.toDF(*[col.upper() for col in df.columns])

In [22]:
df.show(5)
df.printSchema()

+-------------------+------+------+------+------+------+----+-----+---+--------+----------+-------------+------+---------+------+--------------------+-----------+----------+
|           DATETIME|  OPEN|   LOW|  HIGH| CLOSE|VOLUME|YEAR|MONTH|DAY|    TIME|      DATE|BASE_CURRENCY|SOURCE|FREQUENCY|SYMBOL|          AUDIT_TIME|       TYPE| LOAD_DATE|
+-------------------+------+------+------+------+------+----+-----+---+--------+----------+-------------+------+---------+------+--------------------+-----------+----------+
|2024-09-08 00:00:00| 68.13| 68.02| 68.78| 68.54|4284.0|2024|    9|  8|        |2024-09-08|          USD|   FMP|     1day| CLUSD|2024-09-24 18:10:...|COMMODITIES|2024-09-24|
|2024-09-08 00:00:00|2526.5|2523.7|2528.3|2527.4|4639.0|2024|    9|  8|        |2024-09-08|          USD|   FMP|     1day| GCUSD|2024-09-24 18:10:...|COMMODITIES|2024-09-24|
|2024-09-08 00:00:00|  2.22| 2.192| 2.232| 2.199|5596.0|2024|    9|  8|        |2024-09-08|          USD|   FMP|     1day| NGUSD|2

## Append the batch data to RAW table

Set the destination raw table

In [23]:
BUCKET_NAME = "cryptoengineer"
SILVER_PREFIX = "datalake/silver/commodities"

In [24]:
path=f"s3://{BUCKET_NAME}/{SILVER_PREFIX}"
print("Path:",path)

Path: s3://cryptoengineer/datalake/silver/commodities


In [25]:
(
    df
    .repartition("YEAR")
    .write
    .format("parquet")
    .mode("append")
    .partitionBy(['SYMBOL','YEAR'])
    .save(path)
)